In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split    
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten, BatchNormalization,Normalization

2025-01-21 01:13:47.752416: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-21 01:13:47.759309: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-21 01:13:47.767829: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-21 01:13:47.770434: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 01:13:47.777012: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Descargar el conjunto de datos de eficiencia energética de UCI
url = ("https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx")
data = pd.read_excel(url)

In [3]:
y_full = data[['Y1','Y2']].copy()
x_full = data.drop(['Y1','Y2'],axis=1)
x_train_full, x_test, y_train_full, y_test = train_test_split(x_full, y_full, test_size=0.2, random_state=17)
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.2, random_state=17)


In [4]:
learning_rates = [1e-4, 7e-4, 1e-3, 5e-3]
models = []
histories = []


tf.keras.backend.clear_session()
tf.random.set_seed(17)

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(x_train)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)

i = 0
for lr in learning_rates:
    # Definir el modelo
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(8,)),  # Capa de entrada correcta
        norm_layer,  # Normalization aquí, no antes
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dense(2)  # Cambiado a 2 neuronas para clasificación binaria
    ])
    
    # Asegúrate de adaptar la normalización a x_train DESPUÉS de definir el modelo
    
   
    
    # Definir el optimizador con la tasa de aprendizaje actual
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    
    model.compile(loss="mean_absolute_error",
                  optimizer=optimizer,
                  metrics=["mean_absolute_error"])
    
    # Por motivos que escapan a mi comprensión (y a la de chatGPT) si descomentamos esto peta
    #norm_layer.adapt(x_train)


    models.append(model)


UnboundLocalError: local variable 'input_shape' referenced before assignment

In [ ]:
for model in models:
    print("Learning rate:", model.optimizer.learning_rate.numpy())
    histories.append(model.fit(x_train, y_train, epochs=75, validation_data=(x_val, y_val),callbacks=early_stopping_cb))